Rosyiidah Dhiya'Ulhaq (19/446468/TK/49573)

> UAS Teknik Pengolahan Citra Biomedis



**Classification Positive or Negative using Fully Automated
Deep Learning-based Network with Lung CT Image for COVID-19**


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Upgrade dan install essential libraries dari sumber
!pip install git+https://github.com/mr7495/RetinaNet

In [ ]:
from tensorflow import keras
import numpy as np
import cv2
import os
import random
import shutil
import pandas as pd
import csv
import glob
import zipfile
from keras import optimizers
from keras.models import Sequential,Model
from keras.layers import Dropout, Flatten, Dense,Input
from keras.applications.resnet_v2 import ResNet50V2
from keras.applications.xception import Xception
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.callbacks import ModelCheckpoint
from keras.applications.imagenet_utils import preprocess_input
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.initializers import RandomNormal
import keras.backend as k
from sklearn.utils import shuffle
import io
from PIL import Image as pil_image
from keras_retinanet import layers
import keras.backend as k
import keras_retinanet

In [ ]:
shape=(512,512,1) #shape dari citra dataset yang dipakai (dalam format .tiff)

In [ ]:
#CT selection Algorithm
!mkdir 'selected_dataD' #membuat folder baru untuk citra terpilih
data_path='/content/drive/MyDrive/Dataset/Pasien D (Normal)' #folder dimana dataset berada
data_files=[]
for r,d,f in os.walk(data_path):
  for file in f:
    if '.tiff' in file:
      data_files.append(os.path.join(r,file)) #mengetahui images path dari data_path

selected=[]
zero=[]
names=[]
for img_path in data_files:
    names.append(img_path)
    pixel=cv2.imread(img_path,cv2.IMREAD_UNCHANGED ) #membaca file .tiff
    sp=pixel[240:340,120:370] #memotong wilayah citra, agar sistem fokus menganalisis bagian tengah citra
    counted_zero=0
    for i in np.reshape(sp,(sp.shape[0]*sp.shape[1],1)):
        if i<300: #hitung jumlah nilai piksel di wilayah yang kurang dari 300
            counted_zero+=1
    zero.append(counted_zero) #tambahkan jumlah piksel gelap gambar ke daftar
min_zero=min(zero)
max_zero=max(zero)
threshold=(max_zero-min_zero)/1.5 #mengatur nilai threshold
indices=np.where(np.array(zero)>threshold) #menemukan gambar yang memiliki lebih banyak piksel gelap di wilayah tersebut daripada threshold yang terhitung
selected_names=np.array(names)[indices] #image terpilih


for selected_img in selected_names:
    shutil.copy(selected_img,'selected_dataD')

In [ ]:
#The fully automated patient classification network
Path='/content/drive/MyDrive/COVID-CTset/trained_networks/ResNet50V2-fold2.hdf5' #Trained Network
selected_imgs_path=[]
for r,d,f in os.walk('selected_dataD'): #menambahkan folder dari citra terpilih
  for file in f:
    if '.tiff' in file:
      selected_imgs_path.append(os.path.join(r,file)) #mendapatkan images path

k.clear_session() #clear keras backend
custom_object={'UpsampleLike': keras_retinanet.layers._misc.UpsampleLike} #lapisan upsampling yang digunakan dalam FPN
net=keras.models.load_model(Path, custom_objects=custom_object)


covid_label=0 #index dari kelas positif COVID-19
normal_label=1  #index dari kelas negatif COVID-19
normal_predicted=0
covid_predicted=0
for img_name in selected_imgs_path: #read images
  img=cv2.imread(img_name,cv2.IMREAD_UNCHANGED)
  pred_ind=np.argmax(net.predict(np.expand_dims(np.expand_dims(img,axis=0),axis=3))[0]) #predicted label
  if pred_ind==normal_label:
    normal_predicted+=1
  else:
    covid_predicted+=1

all_imgs_num=len(selected_imgs_path)
if covid_predicted>=0.1*all_imgs_num: #jika setidaknya 10% dari CT scan pasien diklasifikasikan sebagai covid
  print('Pasien teridentifikasi Positif COVID-19')
else:
  print('Pasien teridentifikasi Negatif COVID-19')

In [ ]:
#Mengetahui detail klasifikasi citra CT Scan dari pasien positif COVID-19
Path='/content/drive/MyDrive/COVID-CTset/trained_networks/ResNet50V2-fold2.hdf5'
selected_imgs_path=[]
for r,d,f in os.walk('selected_dataA'):
  for file in f:
    if '.tiff' in file:
      selected_imgs_path.append(os.path.join(r,file))

k.clear_session()
custom_object={'UpsampleLike': keras_retinanet.layers._misc.UpsampleLike} #
net=keras.models.load_model(Path, custom_objects=custom_object)


covid_label=0
normal_label=1
normal_predicted=0
covid_predicted=0
for img_name in selected_imgs_path:
  img=cv2.imread(img_name,cv2.IMREAD_UNCHANGED)
  pred_ind=np.argmax(net.predict(np.expand_dims(np.expand_dims(img,axis=0),axis=3))[0])
  if pred_ind==normal_label:
    normal_predicted+=1
    print('Normal') #citra tidak teridentifikasi adanya virus corona
  else:
    covid_predicted+=1
    print('COVID-19')  #citra teridentifikasi adanya virus corona